In [9]:
import os
import pandas

mirna_src_dir = os.getcwd() + "/assn-mirna-luad/data/processed/miRNA/"
clinical_src_dir = os.getcwd() + "/assn-mirna-luad/data/processed/clinical/"

mirna_tumor_df = pandas.read_csv(mirna_src_dir+'tumor_miRNA.csv')
mirna_normal_df = pandas.read_csv(mirna_src_dir+'normal_miRNA.csv')
clinical_df = pandas.read_csv(clinical_src_dir+'clinical.csv')

print "mirna_tumor_df.shape", mirna_tumor_df.shape
print "mirna_normal_df.shape", mirna_normal_df.shape

"""
Here we select samples to use for our regression analysis
"""
matched_samples = pandas.merge(clinical_df, mirna_normal_df, on='patient_barcode')['patient_barcode']
print "matched_samples", matched_samples.shape
# merged = pandas.merge(clinical_df, mirna_normal_df, on='patient_barcode')
# print merged.shape
# print
# print merged['histological_type'].value_counts().sort_index(axis=0)
# print
# print merged['pathologic_stage'].value_counts().sort_index(axis=0)
# print
# print merged['pathologic_T'].value_counts().sort_index(axis=0)
# print
# print merged['pathologic_N'].value_counts().sort_index(axis=0)
# print
# print merged['pathologic_M'].value_counts().sort_index(axis=0)
# print

mirna_tumor_df.shape (513, 1882)
mirna_normal_df.shape (46, 1882)
matched_samples (46,)


## We now create data matrixes from tumor vs normal samples

In [19]:
from sklearn import preprocessing

X_normal = mirna_normal_df[mirna_normal_df['patient_barcode'].isin(matched_samples)]
X_tumor = mirna_tumor_df[mirna_tumor_df['patient_barcode'].isin(matched_samples)]
X_normal.__delitem__('patient_barcode')
X_tumor.__delitem__('patient_barcode')

print "X_normal.shape", X_normal.shape
print "X_tumor.shape", X_tumor.shape

X = pandas.concat([X_normal, X_tumor])

# Create labels for 92 samples, first 46 labeled 0 indicating normal tissues, and last 46 labeled 1 for tumor tissues
Y = pandas.concat([pandas.DataFrame([0,]*46, dtype=int), pandas.DataFrame([1,]*46, dtype=int)])

print 'miRNA null values', X.isnull().sum().sum()
print 'Y null values', Y.isnull().sum()
# print 'X.shape', X.shape
# print 'Y.shape', Y.shape
# mirna_list = X.columns.values
# 
X_scaler = preprocessing.StandardScaler(with_mean=False).fit(X)
X = X_scaler.transform(X)
# # Y_scaler = preprocessing.StandardScaler().fit(Y)
# # Y = Y_scaler.transform(Y)

X_normal.shape (46, 1881)
X_tumor.shape (46, 1881)
miRNA null values 0
Y null values 0    0
dtype: int64


## Perform classification with LASSO

In [20]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=np.random.randint(0, 10000))
print 'X_train', X_train.shape, ', y_train', y_train.shape
print 'X_test', X_test.shape, ', y_test', y_test.shape

model = linear_model.LassoCV(cv=3, fit_intercept=False, normalize=False)
print model.fit(X_train, y_train)

print metrics.confusion_matrix(y_test, model.predict(X_test))

X_train (82, 1881) , y_train (82, 1)
X_test (10, 1881) , y_test (10, 1)


LassoCV(alphas=None, copy_X=True, cv=3, eps=0.001, fit_intercept=False,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)


ValueError: Can't handle mix of binary and continuous